In [1]:
import pandas as pd

In [2]:
import speaker_diarization
from combine_stt_diarization import combine_stt_diarization
from time_locator import ctm_to_groupdata, get_time_span, get_report

### Speaker Diarization

In [3]:
model_file = '/home/jovyan/wm-insur-call-qa/River/xvector_self/備份/total_model_1s'
wav_file = '/home/jovyan/wm-insur-call-qa/testing_data/電訪-房貸壽險-南山.wav'

In [4]:
sd = speaker_diarization.Speaker_Diarization(model_file, wav_file, 10, 0.5, 5, 3)

/project/wm-insur-call-qa/owen/speaker_diarization.py:49: WavFileWarning: Chunk (non-data) not understood, skipping it.
  assert wavfile.read(wav)
/project/wm-insur-call-qa/owen/speaker_diarization.py:49: WavFileWarning: Reached EOF prematurely; finished at 3358720 bytes, expected 3358768 bytes from header.
  assert wavfile.read(wav)
/home/jovyan/.local/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/project/wm-insur-call-qa/owen/speaker_diarization.py:34: WavFileWarning: Chunk (non-data) not understood, skipping it.
  self.rate, self.wav = wavfile.read(wav_file)
/project/wm-insur-call-qa/owen/speaker_diarization.py:34: WavFileWarning: Reached EOF prematurely; finished at 3358

In [5]:
sd_result = sd.speaker_diarization_pipeline()

4199it [00:24, 169.58it/s]
100%|██████████| 4178/4178 [00:00<00:00, 2194040.58it/s]


In [6]:
print(sd_result)

[0, 11.2, 11.6, 12.95, 13.3, 14.1, 15.15, 19.3, 19.75, 20.65, 21.5, 31.5, 32.3, 32.75, 33.35, 35.1, 35.2, 37.7, 38.0, 39.5, 40.0, 40.7, 41.95, 44.05, 44.25, 45.45, 46.3, 48.6, 49.85, 51.15, 51.6, 60.25, 62.3, 64.2, 64.7, 73.7, 74.7, 77.0, 77.5, 82.35, 83.1, 85.0, 85.9, 90.6, 94.25, 96.0, 99.15, 105.3, 107.4, 114.3, 114.6, 121.4, 121.55, 132.2, 132.8, 141.3, 141.9, 148.9, 149.4, 150.3, 150.7, 151.9, 152.4, 157.8, 158.35, 160.35, 160.75, 161.8, 162.85, 168.2, 168.75, 175.1, 176.35, 177.55, 182.75, 183.9, 184.95, 186.25, 188.0, 193.5, 195.45, 196.55, 197.05, 197.95, 198.4, 199.95, 201.4, 203.35, 206.1]


### STT (因STT還沒開發成套件型式，現在只能先讀取處理好的檔案。)

In [68]:
data_path = '/home/jovyan/wm-insur-call-qa/Lala/data/stt_result/eval.ctm'
group_name = '/home/jovyan/wm-insur-call-qa/testing_data/電訪-傳統型外幣-中壽.wav'
stt_df = ctm_to_groupdata(data_path, group_name)

In [69]:
stt_df

,file_name,1,start_time,stay_time,word
0,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,2.94,0.21,您好
1,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,14.85,0.39,喂
2,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,15.60,0.57,您好
3,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,16.41,0.33,您好
4,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,16.74,0.33,請問
...,...,...,...,...,...
452,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,198.72,0.39,謝謝
453,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,199.41,0.48,再見
454,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,200.04,0.27,好
455,/home/jovyan/wm-insur-call-qa/testing_data/電訪-...,1,200.40,0.45,謝謝


In [70]:
time_point = 0
text = ''
text_split = []

for idx in stt_df.index:
    if stt_df.loc[idx, 'start_time'] - time_point > 1 and text:
        text_split.append(text)
        text = ''
    text += stt_df.loc[idx, 'word']+' '
    time_point = stt_df.loc[idx, 'start_time'] + stt_df.loc[idx, 'stay_time']
text_split.append(text)

# for stt_df['word']

In [71]:
import torch
import logging
from bert_typo_correct import inference, load_model_and_tokenizer

tokenizer_dir = '/home/jovyan/if-beautiful-text/owen_dev/if_beautiful_text/cache_dir/bert-base-chinese'
bert_dir = '/home/jovyan/if-beautiful-text/owen_dev/if_beautiful_text/cache_dir/bert-base-chinese'
typo_detect_model_dir = '/home/jovyan/if-beautiful-text/owen_dev/if_beautiful_text/models/typo_detect/typo_detect_step100000_loss0.08619457607467969'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'device: {device}')

tokenizer, detect_model, raw_bert = load_model_and_tokenizer(device, tokenizer_dir, bert_dir, typo_detect_model_dir)
logging.info('Load text data without punctuations')

punctuates = []

pred_texts, detect_indexes = inference(text_split, punctuates, tokenizer, detect_model, raw_bert, device)
text = "".join(pred_texts).split()
stt_df['corr_word'] = text

2020-12-28 17:42:14,394 INFO: device: cuda
2020-12-28 17:42:14,396 INFO: Load bert tokenizer and model
Some weights of the model checkpoint at /home/jovyan/if-beautiful-text/owen_dev/if_beautiful_text/cache_dir/bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2020-12-28 17:42:22,257 INFO: Load text data without punctuations
2020-12-28 17:42:22,258 INFO: Data transformation for model inference
2020-12-28 17:42:22,259 INFO: Run prediction
evaluate: 100%|

In [72]:
with open('correct_result.txt', 'a') as fa:
    fa.write(group_name + '\n')
s1, s2 = '', ''
for idx in stt_df.index:
    if stt_df.loc[idx, 'word'] != stt_df.loc[idx, 'corr_word']:
        s1 += '[' + stt_df.loc[idx, 'word'] + ']'
        s2 += '[' + stt_df.loc[idx, 'corr_word'] + ']'
    else:
        s1 += ' ' + stt_df.loc[idx, 'word'] + ' '
        s2 += ' ' + stt_df.loc[idx, 'corr_word'] + ' '
    if idx % 20 == 19 or idx == len(stt_df):  
        with open('correct_result.txt', 'a') as fa:
            fa.write(s1 + '\n')
            fa.write(s2 + '\n')
        s1, s2 = '', ''

### 整理語者分離及對話內容

In [11]:
combine_result = combine_stt_diarization(sd_result, stt_df[['start_time', 'stay_time', 'word']])

In [12]:
combine_result
#.to_csv('../../if-beautiful-text/owen_dev/if_beautiful_text/src/sttdf.csv', index=False)

,start,end,speaker,word_idx
0,0.00,11.20,0,[喂]
1,11.20,11.60,1,[喂]
2,11.60,12.95,0,"[您好, 這裡, 是, 玉山銀行]"
3,12.95,13.30,1,"[玉山銀行, 新]"
4,13.30,14.10,0,"[新, 北市, 家庭, 中心]"
...,...,...,...,...
84,198.40,199.95,0,"[做, 審核]"
85,199.95,201.40,1,"[也, 會, 再, 通知]"
86,201.40,203.35,0,"[通知, 我, 就, 對, 了, 對, 對, 對, 沒, 錯, OK]"
87,203.35,206.10,1,"[OK, 好, 謝謝, 好, 掰掰]"


### 比對答案

In [13]:
question = ["""您好！這裡是玉山銀行總行個金處/OO分行/OO消金中心，
               敝姓O，員工編號OOOOO，請問是○○○先生/小姐本人嗎？""",
            '感謝您近期透過本行投保○○人壽○○○，繳費年期為O年，依照保險法令的要求，為保障您的權益，稍後電話訪問內容將會全程錄音，請問您同意嗎？'
            '為維護您的資料安全，這裡簡單跟您核對基本資料，您的身分證字號是，請問後三碼是？',
            '請問您的出生年月日是?',
            '請問您是否知道本次購買的是○○人壽的保險，不是存款，如果辦理解約將可能只領回部分已繳保費？',
            '請問您投保時，是否皆由○○消金中心的○○○，在旁邊協助，並由您本人○○○親自簽名，且被保險人之健康告知事項皆由您確認後親自填寫？',
            '請問○○消金中心的○○○是否有向您說明產品內容，並確認符合您的需求？',
            '請問招攬人員是否有提供您一次繳清與分期繳等不同繳費方式選擇？',
            '請問您本次投保繳交保費的資金來源是否為',
            """請問您是否已事先審慎評估自身財務狀況與風險承受能力，
               並願承擔因財務槓桿操作方式所面臨的風險及辦理保單解約轉投保之權益損失，
               除辦理貸款或保單借款需支付本金及利息外，
               還有該產品可能發生之相關風險及最大可能損失，
               且本行人員並未鼓勵或勸誘以辦理貸款、保單借款、保單解約/保單終止及定存解約之方式購買保險，
               請問您是否已瞭解？""",
            '與您確認，本保單之規劃您是否已確實瞭解投保目的、保險需求，並經綜合考量財務狀況以及付費能力，且不影響您的日常支出？',
            '與您再次確認上述投保內容和本次貸款並沒有搭售或不當行銷的情形發生，請問是否正確?',
            '請問您本次辦理貸款及保險，是否有新申請玉山網路銀行？']

In [14]:
result = get_report(speaker_df=combine_result, df=stt_df, questions=question, max_cus_time=10, base_score=0.4)

In [15]:
result

,q_start_time,q_end_time,score,question,recognize_result,reply_start_time,reply_end_time,reply,pass
0,0.00,0.00,0.000000,請問您本次辦理貸款及保險，是否有新申請玉山網路銀行？,,0.00,0.00,[],0
1,11.94,18.57,0.649351,您好！這裡是玉山銀行總行個金處/OO分行/OO消金中心，\n 敝...,您好這裡是玉山銀行新北市家庭中心敝姓劉員工編號是零六六一四好請問是李慧珍小姐本人嗎,19.30,19.75,[嘿我是],1
2,20.52,40.29,0.726316,感謝您近期透過本行投保○○人壽○○○，繳費年期為O年，依照保險法令的要求，為保障您的權益，稍...,感謝你定期透過本行投保發現法國巴黎人壽幸福世代遞減定期受限繳費年輕一次付清依照保險法的要求要...,39.50,44.25,"[身分證字號, 二一零三一零, 七]",0
3,45.45,47.37,1.000000,請問您的出生年月日是?,請問您的出生年月日是,48.60,49.85,[六十一月十七],0
4,51.03,59.22,0.847059,請問您是否知道本次購買的是○○人壽的保險，不是存款，如果辦理解約將可能只領回部分已繳保費？,請問您知道本次購買的是法商法國巴黎人壽保險不是存款如果辦理解約可能只能領回來部分已經繳保費,0.00,0.00,[],0
5,61.47,72.60,0.703704,請問您投保時，是否皆由○○消金中心的○○○，在旁邊協助，並由您本人○○○親自簽名，且被保險人...,請問頭髮時是否新北二心中心的李俊宏先生在旁邊協助並由您本人林惠貞親自簽名車被保險人健康告知事...,73.70,74.70,[是],1
6,75.81,81.15,0.730769,請問○○消金中心的○○○是否有向您說明產品內容，並確認符合您的需求？,中心的李俊豪先生是否向說明產品內容確認符合你的需求,82.35,83.10,[有說明],0
7,83.70,88.74,0.724138,請問招攬人員是否有提供您一次繳清與分期繳等不同繳費方式選擇？,請問江男人員是否提供一次付清還是分期繳不同繳款方式讓您選擇,90.60,94.25,[他說他提供分期分期會比較],0
8,100.80,103.71,0.789474,請問您本次投保繳交保費的資金來源是否為,請問你們是投保繳繳保費的資金來源是否為,0.00,0.00,[],0
9,106.77,140.19,0.788274,請問您是否已事先審慎評估自身財務狀況與風險承受能力，\n 並願承...,請問您是否已事先審慎評估自身的財務狀況風險程式的能力願意承擔財務財務槓桿操作方式所面臨的風險...,141.30,141.90,[了解],1


In [14]:
combine_result

,start,end,speaker,word_idx
0,0.00,11.20,0,[喂]
1,11.20,11.60,1,[喂]
2,11.60,12.95,0,"[您好, 這裡, 是, 玉山銀行]"
3,12.95,13.30,1,"[玉山銀行, 新]"
4,13.30,14.10,0,"[新, 北市, 家庭, 中心]"
...,...,...,...,...
84,198.40,199.95,0,"[做, 審核]"
85,199.95,201.40,1,"[也, 會, 再, 通知]"
86,201.40,203.35,0,"[通知, 我, 就, 對, 了, 對, 對, 對, 沒, 錯, OK]"
87,203.35,206.10,1,"[OK, 好, 謝謝, 好, 掰掰]"


In [25]:
combine_result[45:50]

,start,end,speaker,word_idx,speaktime
45,96.00,99.15,1,"[就, 一, 次, 繳, 然後, 到時候, 就是, 銀行, 這邊]",3.15
46,99.15,105.30,0,"[這邊, 對, 對, 了解, 請問, 你們, 是, 投保, 繳, 繳, 保費, 的, 資金,...",6.15
47,105.30,107.40,1,"[是, 請問, 您, 是否]",2.10
48,107.40,114.30,0,"[是否, 已, 事先, 審慎, 評估, 自身, 的, 財務, 狀況, 風險, 程式, 的, ...",6.90
49,114.30,114.60,1,"[方式, 所]",0.30


In [21]:
for rr in combine_result[30:31]['word_idx']:
    print(rr)

['知道', '本次', '購買', '的', '是', '法商', '法國', '巴黎', '人壽', '保險', '不是', '存款', '如果', '辦理', '解約', '可能', '只', '能領', '回來', '部分', '已經', '繳', '保費']


In [23]:
q_num = []
q_start_time = []
q_end_time = []
q_max_score = []
q_token = []
question_answer = []
for i in range(len(question)):
    result2 = get_time_span(stt_df, question[i])
    q_num.append('Q_'+str(i+1))
    q_start_time.append(result2['start_time'])
    q_end_time.append(result2['end_time'])
    q_max_score.append(result2['max_score'])
    q_token.append(''.join(result2['tokens_text']))
    question_answer.append(question[i])
q_result_dict = {'question_num': q_num,
                 'question_start_time': q_start_time,
                 'question_end_time': q_end_time,
                 'question_max_score': q_max_score,
                 'q_token': q_token,
                 'question_answer': question_answer}

In [24]:
pd.DataFrame(q_result_dict)

,question_num,question_start_time,question_end_time,question_max_score,q_token,question_answer
0,Q_1,11.94,18.57,0.649351,您好這裡是玉山銀行新北市家庭中心敝姓劉員工編號是零六六一四好請問是李慧珍小姐本人嗎,您好！這裡是玉山銀行總行個金處/OO分行/OO消金中心，\n 敝...
1,Q_2,20.52,40.29,0.726316,感謝你定期透過本行投保發現法國巴黎人壽幸福世代遞減定期受限繳費年輕一次付清依照保險法的要求要...,感謝您近期透過本行投保○○人壽○○○，繳費年期為O年，依照保險法令的要求，為保障您的權益，稍...
2,Q_3,45.45,47.37,1.000000,請問您的出生年月日是,請問您的出生年月日是?
3,Q_4,51.03,59.22,0.847059,請問您知道本次購買的是法商法國巴黎人壽保險不是存款如果辦理解約可能只能領回來部分已經繳保費,請問您是否知道本次購買的是○○人壽的保險，不是存款，如果辦理解約將可能只領回部分已繳保費？
4,Q_5,61.47,72.60,0.703704,請問頭髮時是否新北二心中心的李俊宏先生在旁邊協助並由您本人林惠貞親自簽名車被保險人健康告知事...,請問您投保時，是否皆由○○消金中心的○○○，在旁邊協助，並由您本人○○○親自簽名，且被保險人...
5,Q_6,75.81,81.15,0.730769,中心的李俊豪先生是否向說明產品內容確認符合你的需求,請問○○消金中心的○○○是否有向您說明產品內容，並確認符合您的需求？
6,Q_7,83.70,88.74,0.724138,請問江男人員是否提供一次付清還是分期繳不同繳款方式讓您選擇,請問招攬人員是否有提供您一次繳清與分期繳等不同繳費方式選擇？
7,Q_8,100.80,103.71,0.789474,請問你們是投保繳繳保費的資金來源是否為,請問您本次投保繳交保費的資金來源是否為
8,Q_9,106.77,140.19,0.788274,請問您是否已事先審慎評估自身的財務狀況風險程式的能力願意承擔財務財務槓桿操作方式所面臨的風險...,請問您是否已事先審慎評估自身財務狀況與風險承受能力，\n 並願承...
9,Q_10,142.59,150.84,0.762887,確認保單規劃也去瞭解投保目的保險需求已經綜合考量財務狀況付費能力並不會影響你的日常生活支出,與您確認，本保單之規劃您是否已確實瞭解投保目的、保險需求，並經綜合考量財務狀況以及付費能力，...
